In [1]:
# 필요한 라이브러리 불러오기

import pandas as pd

In [2]:
folder_path = "C:/Users/SSAFY/Desktop/git-repos/time-traveler/add-tools/data"

In [3]:
total_spots = pd.read_csv(f"{folder_path}/total_info_for_embedding.csv", encoding="utf-8")
print(total_spots.shape)
total_spots.head(2)

(519, 12)


,spot_code,name,category,address,mapx,mapy,tel,homepage,visitkorea_page,original_image,overview,preprocessed_reviews
0,2718737,트릭아트스토리,문화시설,인천광역시 중구 동화마을길 42 (송월동3가),126.620739,37.477686,032-761-2631,https://itour.incheon.go.kr/ssst/ssst/detail.d...,https://korean.visitkorea.or.kr/detail/ms_deta...,https://itour.incheon.go.kr/common/viewImg.do?...,송월동 동화마을에 있는 트릭아트스토리는 관객 참여형 놀이공간인 트릭아트 전시관이다....,NaN
1,130105,가천박물관,문화시설,인천광역시 연수구 청량로102번길 40-9,126.657604,37.418224,032-833-4747,http://gachonmuseum.or.kr/,https://korean.visitkorea.or.kr/detail/ms_deta...,NaN,"가천박물관은 1995년 10월 문을 연 가천문화재단 부속 박물관으로, 국내 사립박물...",NaN


In [4]:
total_spots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   spot_code             519 non-null    int64  
 1   name                  519 non-null    object 
 2   category              519 non-null    object 
 3   address               518 non-null    object 
 4   mapx                  519 non-null    float64
 5   mapy                  519 non-null    float64
 6   tel                   466 non-null    object 
 7   homepage              386 non-null    object 
 8   visitkorea_page       477 non-null    object 
 9   original_image        387 non-null    object 
 10  overview              518 non-null    object 
 11  preprocessed_reviews  139 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 48.8+ KB


In [9]:
total_spots["preprocessed_reviews"][15]

'책 읽기 쾌적해요(12)\n독특한 책이 많아요(11)\n카페 같은 분위기에요(11)\n책 종류가 다양해요(9)\n아늑해요(5)\n인테리어가 멋져요(5)\n베스트셀러가 많아요(3)\n컨셉이 독특해요(3)\n원하는 책을 찾기 쉬워요(2)\n시설이 깔끔해요(2)'

## CSV 로더를 이용해서 document 생성하기

In [5]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path=f"{folder_path}/total_info_for_embedding.csv",
    source_column="name",
    metadata_columns=["spot_code", "mapx", "mapy", "tel", "homepage", "visitkorea_page", "original_image"],
    encoding="utf-8",
)

docs = loader.load()

print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

519
name: 트릭아트스토리
category: 문화시설
address: 인천광역시 중구 동화마을길 42 (송월동3가)
overview: 송월동 동화마을에 있는 트릭아트스토리는 관객 참여형 놀이공간인 트릭아트 전시관이다. 트릭아트스토리는 2층 규모로 2015년 개관했다. 송월동 동화마을 안에 있어, 동화마을의 인기 관광 코스이다. 이곳에는 45점의 트릭아트 벽화가 전시돼 있다. 트릭아트란 과학적인 화법과 특수 도료를 이용해 그림이 마치 살아 움직이는 것처럼 느껴지도록 입체적으로 그린 그림이다. 관람객들은 단순히 그림을 감상하는 것이 아니라 입체 그림에 들어가서 포즈를 취하고 사진을 찍는 등 재미있게 체험할 수 있다. 트릭아트 외에도 동작 인식 시스템과 거울미로, 블랙 아트 등 흥미로운 전시물이 있다.
preprocessed_reviews: 
{'source': '트릭아트스토리', 'row': 0, 'spot_code': '2718737', 'mapx': '126.6207395', 'mapy': '37.47768599', 'tel': '032-761-2631', 'homepage': 'https://itour.incheon.go.kr/ssst/ssst/detail.do?cotId=ITD21122911322754938#guide', 'visitkorea_page': 'https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=589200a6-9a84-450c-9711-1806c798e408', 'original_image': 'https://itour.incheon.go.kr/common/viewImg.do?imgId=DBI21122911353909342'}


### 임베딩

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_API_KEY")

In [7]:
# 이건 그냥 로컬에서 안됨
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

c:\Users\SSAFY\Desktop\git-repos\time-traveler\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load model directly - X
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")

c:\Users\SSAFY\Desktop\git-repos\time-traveler\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SSAFY\Desktop\git-repos\time-traveler\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--intfloat--multilingual-e5-large-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")

In [8]:
# csv 로더를 통해 문서화한 page_content를 하나로 모으기
total_content = []

for doc in docs:
    total_content.append(doc.page_content)

total_content[0]

'name: 트릭아트스토리\ncategory: 문화시설\naddress: 인천광역시 중구 동화마을길 42 (송월동3가)\noverview: 송월동 동화마을에 있는 트릭아트스토리는 관객 참여형 놀이공간인 트릭아트 전시관이다. 트릭아트스토리는 2층 규모로 2015년 개관했다. 송월동 동화마을 안에 있어, 동화마을의 인기 관광 코스이다. 이곳에는 45점의 트릭아트 벽화가 전시돼 있다. 트릭아트란 과학적인 화법과 특수 도료를 이용해 그림이 마치 살아 움직이는 것처럼 느껴지도록 입체적으로 그린 그림이다. 관람객들은 단순히 그림을 감상하는 것이 아니라 입체 그림에 들어가서 포즈를 취하고 사진을 찍는 등 재미있게 체험할 수 있다. 트릭아트 외에도 동작 인식 시스템과 거울미로, 블랙 아트 등 흥미로운 전시물이 있다.\npreprocessed_reviews: '

In [9]:
%%time

# document 임베딩
# embedded_documents = model.encode(total_content)    # sentence transformer 사용
embedded_documents = hf_embeddings.embed_documents(total_content)

print(len(embedded_documents[0]))

1024
CPU times: total: 57min 17s
Wall time: 4min 15s


In [10]:
# 테스트
query = "트릭아트스토리에 대해서 알려줘."

# embedded_query = model.encode(query)
embedded_query = hf_embeddings.embed_query(query)

print(embedded_query)

[-0.0010814785491675138, 0.0050503253005445, -0.014546170830726624, -0.027664821594953537, 0.03407861292362213, -0.02239232510328293, -0.03346359357237816, 0.02410353161394596, 0.0648617371916771, -0.023566443473100662, 0.037761982530355453, 0.01125197671353817, -0.02282237820327282, -0.0016356495907530189, -0.021656598895788193, -0.023532956838607788, -0.042411912232637405, 0.008973202668130398, 0.0038503985852003098, -0.0066417548805475235, 0.0383797362446785, -0.010445966385304928, -0.02681824564933777, -0.03280091658234596, -0.026901336386799812, -0.024466222152113914, -0.01864476688206196, -0.026080090552568436, -0.00436973525211215, -0.03809821233153343, -0.002393190748989582, 0.014342443086206913, -0.04526413232088089, -0.04788554832339287, -0.02129400707781315, 0.018500035628676414, 0.059725452214479446, 0.014975802041590214, -0.022127509117126465, 0.05656171590089798, -0.00915971677750349, 0.030604802072048187, 0.030824918299913406, 0.01327677071094513, -0.045157309621572495, 

### 유사도 계산

In [11]:
import numpy as np

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

for i, idx in enumerate(sorted_idx[:3]):
    print(f"{i+1}. {total_content[idx]}")

1. name: 트릭아트스토리
category: 문화시설
address: 인천광역시 중구 동화마을길 42 (송월동3가)
overview: 송월동 동화마을에 있는 트릭아트스토리는 관객 참여형 놀이공간인 트릭아트 전시관이다. 트릭아트스토리는 2층 규모로 2015년 개관했다. 송월동 동화마을 안에 있어, 동화마을의 인기 관광 코스이다. 이곳에는 45점의 트릭아트 벽화가 전시돼 있다. 트릭아트란 과학적인 화법과 특수 도료를 이용해 그림이 마치 살아 움직이는 것처럼 느껴지도록 입체적으로 그린 그림이다. 관람객들은 단순히 그림을 감상하는 것이 아니라 입체 그림에 들어가서 포즈를 취하고 사진을 찍는 등 재미있게 체험할 수 있다. 트릭아트 외에도 동작 인식 시스템과 거울미로, 블랙 아트 등 흥미로운 전시물이 있다.
preprocessed_reviews: 
2. name: 삼랑성역사문화축제
category: 축제
address: 인천광역시 강화군 길상면 전등사로 37-41
overview: 전등사를 둘러싼 삼랑성은 외적의 침입을 막기위해 단군의 세아들이 쌓았다는 전설이 있다. 고려때 몽골의 침략을 피해 강화도로 천도한 뒤 삼랑성 가궐에서 항몽의지를 불태우며 120일 동안 대불정오설도량을 펼친 곳이기도 하다. 또한, 조선왕조실록을 250년 동안 보관했던 곳이기도 하며, 프랑스 군을 격퇴한 병인양요의 승전지다. 뿐 만 아니라 구한말 일제에 항거하며 1주일 동안 의병전투에서 승리한 곳이기도 하다. 5,000년 간 우리역사와 끊임없이 교류하고, 우리 역사를 품은 민족자존의 현장이다.
우리의 자랑스런 역사의 자긍심을 느끼고, 널리 알리기 위해 삼랑성역사문화축제를 매년 10월 첫 주 토요일부터 개최하여 올해 24회를 맞는다.
preprocessed_reviews: 
3. name: 시민창작예술축제 학산마당극놀래
category: 공연/행사
address: 인천광역시 미추홀구 경인로 216-1 (도화동)
overview: 이웃과 삶을 나누는 이야기 신명마당, 마을에서 모이고 배우고 놀아보

In [16]:
from numpy.linalg import norm

for i, idx in enumerate(sorted_idx[:3]):
    doc_vector = embedded_documents[idx]
    sim = np.dot(embedded_query, doc_vector) / (norm(embedded_query) * norm(doc_vector))
    print(f"[{i+1}] score : {sim}\n{total_content[idx]}")

[1] score : 0.8575432896614075
name: 트릭아트스토리
category: 문화시설
address: 인천광역시 중구 동화마을길 42 (송월동3가)
overview: 송월동 동화마을에 있는 트릭아트스토리는 관객 참여형 놀이공간인 트릭아트 전시관이다. 트릭아트스토리는 2층 규모로 2015년 개관했다. 송월동 동화마을 안에 있어, 동화마을의 인기 관광 코스이다. 이곳에는 45점의 트릭아트 벽화가 전시돼 있다. 트릭아트란 과학적인 화법과 특수 도료를 이용해 그림이 마치 살아 움직이는 것처럼 느껴지도록 입체적으로 그린 그림이다. 관람객들은 단순히 그림을 감상하는 것이 아니라 입체 그림에 들어가서 포즈를 취하고 사진을 찍는 등 재미있게 체험할 수 있다. 트릭아트 외에도 동작 인식 시스템과 거울미로, 블랙 아트 등 흥미로운 전시물이 있다.
preprocessed_reviews: 
[2] score : 0.8175342082977295
name: 삼랑성역사문화축제
category: 축제
address: 인천광역시 강화군 길상면 전등사로 37-41
overview: 전등사를 둘러싼 삼랑성은 외적의 침입을 막기위해 단군의 세아들이 쌓았다는 전설이 있다. 고려때 몽골의 침략을 피해 강화도로 천도한 뒤 삼랑성 가궐에서 항몽의지를 불태우며 120일 동안 대불정오설도량을 펼친 곳이기도 하다. 또한, 조선왕조실록을 250년 동안 보관했던 곳이기도 하며, 프랑스 군을 격퇴한 병인양요의 승전지다. 뿐 만 아니라 구한말 일제에 항거하며 1주일 동안 의병전투에서 승리한 곳이기도 하다. 5,000년 간 우리역사와 끊임없이 교류하고, 우리 역사를 품은 민족자존의 현장이다.
우리의 자랑스런 역사의 자긍심을 느끼고, 널리 알리기 위해 삼랑성역사문화축제를 매년 10월 첫 주 토요일부터 개최하여 올해 24회를 맞는다.
preprocessed_reviews: 
[3] score : 0.8153867125511169
name: 시민창작예술축제 학산마당극놀래
category: 공연/행사

## vectordb 만들기

In [12]:
# 벡터DB 넣을 폴더
DB_PATH = "C:/Users/SSAFY/Desktop/git-repos/time-traveler/vectordb"

In [13]:
from langchain_chroma import Chroma

db = Chroma.from_documents(
    documents=docs,
    embedding=hf_embeddings,
    persist_directory=DB_PATH + "/spot_db_in_local",
    collection_name="spot_db"
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
